# Imports

Libraries used are tensorflow 2.0, nunmpy, and matplotlib

In [1]:
import numpy as np
import matplotlib as mpl
import tensorflow as tf

from matplotlib import pyplot as plt
from tensorflow import keras

In [2]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# Dataset

In [3]:
from tensorflow.keras.datasets import mnist

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

In [4]:
print('MNIST Dataset Shape:')
print('X_train: ' + str(X_train.shape))
print('Y_train: ' + str(Y_train.shape))
print('X_test:  '  + str(X_test.shape))
print('Y_test:  '  + str(Y_test.shape))

MNIST Dataset Shape:
X_train: (60000, 28, 28)
Y_train: (60000,)
X_test:  (10000, 28, 28)
Y_test:  (10000,)


# Discriminator

In [5]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

def create_discriminator_model():
    model = Sequential()
    model.add(Conv2D(128, (3, 3), activation='relu', input_shape=(28, 28, 1), padding='same'))
    model.add(MaxPooling2D(2, 2))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(2, 2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    return model

In [6]:
discriminator_model = create_discriminator_model()
# discriminator_model.compile(optimizer='adam', loss='binary_crossentropy')

# Generator

In [7]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import Conv2DTranspose

dim = 7
depth = 256

def create_generator_model():
    model = Sequential()
    model.add(Dense(dim * dim * depth, activation='relu', input_dim=100))
    model.add(Reshape((dim, dim, depth)))
    model.add(Dropout(0.2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, 3, padding='same'))
    model.add(Activation('relu'))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(1, 3, padding='same'))
    model.add(Activation('relu'))

    return model

In [8]:
generator_model = create_generator_model()

# create weights with the right shape, e.g.
weights = [np.random.rand(*w.shape) for w in generator_model.get_weights()]

# update
generator_model.set_weights(weights)

# generator_model.compile(optimizer='adam', loss='binary_crossentropy')

# Training

Define constants

In [9]:
batch_size = 10
epoch_steps = len(X_train) // batch_size * 2

Get a random batch from the real numbers

In [10]:
def get_real_batch(size):
    X = X_train[np.random.choice(X_train.shape[0], size, replace=False), :]
    return X

In [11]:
def get_batch(size, generator):
    # In case the batch size is odd
    generator_batch_size = size // 2
    real_batch_size = size - generator_batch_size

    rand_vals = np.random.rand(generator_batch_size, 100)
    generator_batch = generator.predict(x=rand_vals, batch_size=generator_batch_size, verbose=2)
    real_batch_size = get_real_batch(real_batch_size)

    batch = np.concatenate((generator_batch, real_batch_size))

    return batch, np.concatenate((np.zeros(generator_batch_size), np.ones(real_batch_size)))

**Training the discriminator**

Code is taken from https://keras.io/guides/writing_a_training_loop_from_scratch/.

In [12]:
# Instantiate an optimizer.
optimizer = keras.optimizers.SGD(learning_rate=1e-3)
# Instantiate a loss function.
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Prepare the training dataset.
batch_size = 64
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = np.reshape(x_train, (-1, 784))
x_test = np.reshape(x_train, (-1, 784))
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

In [ ]:
epochs = 2
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))

    # Iterate over the batches of the dataset.
    for step in range(epoch_steps):

        x_batch_train, y_batch_train = get_batch(batch_size, generator_model)

        # Open a GradientTape to record the operations run
        # during the forward pass, which enables autodifferentiation.
        with tf.GradientTape() as tape:

            # Run the forward pass of the layer.
            # The operations that the layer applies
            # to its inputs are going to be recorded
            # on the GradientTape.
            logits = discriminator_model(x_batch_train, training=True)  # Logits for this minibatch

            # Compute the loss value for this minibatch.
            loss_value = loss_fn(y_batch_train, logits)

        # Use the gradient tape to automatically retrieve
        # the gradients of the trainable variables with respect to the loss.
        grads = tape.gradient(loss_value, discriminator_model.trainable_weights)

        # Run one step of gradient descent by updating
        # the value of the variables to minimize the loss.
        optimizer.apply_gradients(zip(grads, discriminator_model.trainable_weights))

        # Log every 200 batches.
        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %s samples" % ((step + 1) * 64))



Start of epoch 0
